# Importer les packages nécessaire pour la réalisation de la question 1

In [72]:
import os
from shutil import copyfile
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader
from question1 import separate_train_test
import torch.optim as optim

from poutyne import set_seeds, Model, ModelCheckpoint, CSVLogger, ReduceLROnPlateau

In [73]:
def get_normalize(use_data = True) :
    
    """calcul la moyenne et l'écart-type d'un jeu de données
    
    Params : 
    use data : vaut True si on va calculer la moyenne et l'écart-type à partir du dataset et Flase si on va
                retourner la moyenne et l'écart-type de ImageNet
    returne : moyenne et l'écart-type 
    
    """

    if use_data :
        train_path = "./data/train"
        train_data = ImageFolder(train_path, T.Compose([T.Resize([224,224]), T.ToTensor()]))

        nb_samples = len(train_data)

        loader = DataLoader(
            train_data,
            batch_size=nb_samples,
            num_workers=0,
            shuffle=False
        )

        mean = 0.
        std = 0.
        for data in loader:
            batch_samples = data[0].shape[0]
            data = data[0].view(batch_samples, data[0].shape[1], -1)
            mean += data.mean(2).sum(0)
            std += data.std(2).sum(0)
            nb_samples += batch_samples

        mean /= nb_samples
        std /= nb_samples

        normalize = T.Normalize(
            mean=mean,
            std=std)

        return normalize

    else :
        normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        return normalize

Séparer les données en test et train en utilsant la méthode separate_train_test() fournie avec l'énoncé

In [74]:
dataset_path = "./data/images"
train_path = "./data/train"
test_path = "./data/test"

separate_train_test(dataset_path, train_path, test_path)


In [49]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

num_classes = 200
batch_size = 64
learning_rate = 0.01
n_epoch = 20

In [75]:
def get_callbacks(save_path) :
    save_path = save_path 
    os.makedirs(save_path, exist_ok=True)

    best_checkpoint = ModelCheckpoint(os.path.join(save_path, 'best_epoch_{epoch}.ckpt'), monitor='val_acc', 
                        mode='max', save_best_only=True, restore_best=True, verbose=True)

    scheduler = ReduceLROnPlateau(monitor='val_acc', mode='max', patience=3, factor=0.5, verbose=True)

    callbacks = [best_checkpoint, scheduler]
    
    return callbacks

# Utiliser les valeurs du dataset d’entraînement pour normaliser les données

In [76]:
normalize = get_normalize()

train_data = ImageFolder(
    train_path,
    T.Compose([
        T.Resize([224,224]),
        T.ToTensor(),
        normalize,
    ]))

test_data = ImageFolder(
    test_path,
    T.Compose([
        T.Resize([224,224]),
        T.ToTensor(),
        normalize,
    ]))

train_loader = DataLoader(train_data, batch_size, num_workers=0, shuffle=True)
test_loader = DataLoader(test_data, batch_size, num_workers=0)

## Initialisation aléatoire par défaut

In [52]:
resnet18 = models.resnet18()
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

In [53]:
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
callbacks = get_callbacks('default_init_norm_data')

model = Model(resnet18, optimizer, loss_function, batch_metrics=['accuracy'],  device=device)

model.fit_generator(train_loader, train_loader, epochs=n_epoch, callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/20 Train steps: 48 Val steps: 48 988.80s loss: 5.965289 acc: 0.560501 val_loss: 5.273394 val_acc: 0.560501
Epoch 1: val_acc improved from -inf to 0.56050, saving file to default_init_norm_data/best_epoch_1.ckpt
Epoch:  2/20 Train steps: 48 Val steps: 48 1000.78s loss: 5.256517 acc: 0.956149 val_loss: 5.182164 val_acc: 1.582591
Epoch 2: val_acc improved from 0.56050 to 1.58259, saving file to default_init_norm_data/best_epoch_2.ckpt
Epoch:  3/20 Train steps: 48 Val steps: 48 940.47s loss: 5.149596 acc: 1.615562 val_loss: 4.992628 val_acc: 2.505770
Epoch 3: val_acc improved from 1.58259 to 2.50577, saving file to default_init_norm_data/best_epoch_3.ckpt
Epoch:  4/20 Train steps: 48 Val steps: 48 1037.12s loss: 5.021648 acc: 1.747445 val_loss: 4.918985 val_acc: 2.472799
Epoch:  5/20 Train steps: 48 Val steps: 48 1041.82s loss: 4.925502 acc: 2.011210 val_loss: 4.791216 val_acc: 3.198154
Epoch 5: val_acc improved from 2.50577 to 3.19815, saving file to default_init_norm_data/best_

## Modèle pré-entraîné, mais en gelant tous les paramètres de convolution

In [54]:
def freeze_conv(resnet18):
    for name, param in resnet18.named_parameters():
        if (name.find('conv') != -1) or (name.find('bn') != -1):
            param.requires_grad = False

In [55]:
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
freeze_conv(resnet18)

In [56]:
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
callbacks = get_callbacks('freeze_conv_norm_data')

model = Model(resnet18, optimizer, loss_function, batch_metrics=['accuracy'],  device=device)

model.fit_generator(train_loader, train_loader, epochs=n_epoch, callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/20 Train steps: 48 Val steps: 48 609.61s loss: 5.426759 acc: 15.397296 val_loss: 2.662295 val_acc: 38.806462
Epoch 1: val_acc improved from -inf to 38.80646, saving file to freeze_conv_norm_data/best_epoch_1.ckpt
Epoch:  2/20 Train steps: 48 Val steps: 48 610.95s loss: 1.703401 acc: 57.467854 val_loss: 0.954294 val_acc: 76.063304
Epoch 2: val_acc improved from 38.80646 to 76.06330, saving file to freeze_conv_norm_data/best_epoch_2.ckpt
Epoch:  3/20 Train steps: 48 Val steps: 48 610.54s loss: 0.712160 acc: 80.184636 val_loss: 0.345212 val_acc: 91.493571
Epoch 3: val_acc improved from 76.06330 to 91.49357, saving file to freeze_conv_norm_data/best_epoch_3.ckpt
Epoch:  4/20 Train steps: 48 Val steps: 48 611.28s loss: 0.290794 acc: 91.955160 val_loss: 0.180804 val_acc: 95.483020
Epoch 4: val_acc improved from 91.49357 to 95.48302, saving file to freeze_conv_norm_data/best_epoch_4.ckpt
Epoch:  5/20 Train steps: 48 Val steps: 48 611.00s loss: 0.155709 acc: 95.911639 val_loss: 0.059

## Modèle pré-entraîné, mais en gelant uniquement les paramètres dans "layer1".

In [57]:
def freeze_layer1(resnet18):
    for name, param in resnet18.named_parameters():
        if name.startswith('layer1'):
            param.requires_grad = False      

In [58]:
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
freeze_layer1(resnet18)

In [59]:
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
callbacks = get_callbacks('freeze_layer1_norm_data')

model = Model(resnet18, optimizer, loss_function, batch_metrics=['accuracy'],  device=device)

model.fit_generator(train_loader, train_loader, epochs=n_epoch, callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/20 Train steps: 48 Val steps: 48 896.63s loss: 6.491557 acc: 0.791296 val_loss: 5.421585 val_acc: 0.791296
Epoch 1: val_acc improved from -inf to 0.79130, saving file to freeze_layer1_norm_data/best_epoch_1.ckpt
Epoch:  2/20 Train steps: 48 Val steps: 48 893.45s loss: 5.344504 acc: 0.494560 val_loss: 5.243628 val_acc: 0.890208
Epoch 2: val_acc improved from 0.79130 to 0.89021, saving file to freeze_layer1_norm_data/best_epoch_2.ckpt
Epoch:  3/20 Train steps: 48 Val steps: 48 892.48s loss: 5.219238 acc: 0.989120 val_loss: 5.144088 val_acc: 1.351797
Epoch 3: val_acc improved from 0.89021 to 1.35180, saving file to freeze_layer1_norm_data/best_epoch_3.ckpt
Epoch:  4/20 Train steps: 48 Val steps: 48 894.67s loss: 5.131808 acc: 1.351797 val_loss: 5.017112 val_acc: 1.879327
Epoch 4: val_acc improved from 1.35180 to 1.87933, saving file to freeze_layer1_norm_data/best_epoch_4.ckpt
Epoch:  5/20 Train steps: 48 Val steps: 48 896.06s loss: 5.060400 acc: 1.285856 val_loss: 4.955073 val_

## modèle pré-entraîné, mais en laissant tous les paramètres se faire ajuster par backprop

In [77]:
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

In [78]:
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
callbacks = get_callbacks('no_freeze_norm_data')

model = Model(resnet18, optimizer, loss_function, batch_metrics=['accuracy'],  device=device)

model.fit_generator(train_loader, train_loader, epochs=n_epoch, callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/20 Train steps: 48 Val steps: 48 959.66s loss: 6.317790 acc: 0.593472 val_loss: 15.611239 val_acc: 0.494560
Epoch 1: val_acc improved from -inf to 0.49456, saving file to no_freeze_norm_data/best_epoch_1.ckpt
Epoch:  2/20 Train steps: 48 Val steps: 48 924.70s loss: 5.319777 acc: 1.153973 val_loss: 5.156312 val_acc: 1.351797
Epoch 2: val_acc improved from 0.49456 to 1.35180, saving file to no_freeze_norm_data/best_epoch_2.ckpt
Epoch:  3/20 Train steps: 48 Val steps: 48 926.21s loss: 5.158055 acc: 0.989120 val_loss: 5.210630 val_acc: 1.055061
Epoch:  4/20 Train steps: 48 Val steps: 48 927.75s loss: 5.072858 acc: 1.549621 val_loss: 4.974369 val_acc: 2.406858
Epoch 4: val_acc improved from 1.35180 to 2.40686, saving file to no_freeze_norm_data/best_epoch_4.ckpt
Epoch:  5/20 Train steps: 48 Val steps: 48 929.22s loss: 5.011436 acc: 2.011210 val_loss: 4.970963 val_acc: 2.505770
Epoch 5: val_acc improved from 2.40686 to 2.50577, saving file to no_freeze_norm_data/best_epoch_5.ckpt
E

# Utiliser les valeurs de ImageNet pour normaliser les données

In [63]:
normalize = get_normalize(False)

train_data = ImageFolder(
    train_path,
    T.Compose([
        T.Resize([224,224]),
        T.ToTensor(),
        normalize,
    ]))

test_data = ImageFolder(
    test_path,
    T.Compose([
        T.Resize([224,224]),
        T.ToTensor(),
        normalize,
    ]))

train_loader = DataLoader(train_data, batch_size, num_workers=0, shuffle=True)
test_loader = DataLoader(test_data, batch_size, num_workers=0)

## Initialisation aléatoire par défaut

In [64]:
resnet18 = models.resnet18()
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

In [65]:
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
callbacks = get_callbacks('default_init_norm_imagenet')

model = Model(resnet18, optimizer, loss_function, batch_metrics=['accuracy'],  device=device)

model.fit_generator(train_loader, train_loader, epochs=n_epoch, callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/20 Train steps: 48 Val steps: 48 965.24s loss: 5.794710 acc: 0.626442 val_loss: 5.683331 val_acc: 0.824266
Epoch 1: val_acc improved from -inf to 0.82427, saving file to default_init_norm_imagenet/best_epoch_1.ckpt
Epoch:  2/20 Train steps: 48 Val steps: 48 984.02s loss: 5.251793 acc: 1.055061 val_loss: 5.156649 val_acc: 1.846357
Epoch 2: val_acc improved from 0.82427 to 1.84636, saving file to default_init_norm_imagenet/best_epoch_2.ckpt
Epoch:  3/20 Train steps: 48 Val steps: 48 1068.85s loss: 5.153574 acc: 1.648533 val_loss: 5.038213 val_acc: 2.077151
Epoch 3: val_acc improved from 1.84636 to 2.07715, saving file to default_init_norm_imagenet/best_epoch_3.ckpt
Epoch:  4/20 Train steps: 48 Val steps: 48 985.35s loss: 5.074416 acc: 1.648533 val_loss: 4.965138 val_acc: 2.307946
Epoch 4: val_acc improved from 2.07715 to 2.30795, saving file to default_init_norm_imagenet/best_epoch_4.ckpt
Epoch:  5/20 Train steps: 48 Val steps: 48 985.51s loss: 5.001888 acc: 2.011210 val_loss: 

## Modèle pré-entraîné, mais en gelant tous les paramètres de convolution

In [66]:
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
freeze_conv(resnet18)

In [67]:
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
callbacks = get_callbacks('freeze_conv_norm_imagenet')

model = Model(resnet18, optimizer, loss_function, batch_metrics=['accuracy'],  device=device)

model.fit_generator(train_loader, train_loader, epochs=n_epoch, callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/20 Train steps: 48 Val steps: 48 639.38s loss: 5.578110 acc: 16.287504 val_loss: 2.387543 val_acc: 45.070887
Epoch 1: val_acc improved from -inf to 45.07089, saving file to freeze_conv_norm_imagenet/best_epoch_1.ckpt
Epoch:  2/20 Train steps: 48 Val steps: 48 637.10s loss: 1.766285 acc: 57.171118 val_loss: 1.016591 val_acc: 74.645565
Epoch 2: val_acc improved from 45.07089 to 74.64557, saving file to freeze_conv_norm_imagenet/best_epoch_2.ckpt
Epoch:  3/20 Train steps: 48 Val steps: 48 642.66s loss: 0.840001 acc: 78.272338 val_loss: 0.322988 val_acc: 92.383778
Epoch 3: val_acc improved from 74.64557 to 92.38378, saving file to freeze_conv_norm_imagenet/best_epoch_3.ckpt
Epoch:  4/20 Train steps: 48 Val steps: 48 618.21s loss: 0.339522 acc: 91.526541 val_loss: 0.123208 val_acc: 97.461259
Epoch 4: val_acc improved from 92.38378 to 97.46126, saving file to freeze_conv_norm_imagenet/best_epoch_4.ckpt
Epoch:  5/20 Train steps: 48 Val steps: 48 609.77s loss: 0.153728 acc: 96.340257

## Modèle pré-entraîné, mais en gelant uniquement les paramètres dans "layer1"

In [68]:
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
freeze_layer1(resnet18)

In [69]:
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
callbacks = get_callbacks('freeze_layer1_norm_imagenet')

model = Model(resnet18, optimizer, loss_function, batch_metrics=['accuracy'],  device=device)

model.fit_generator(train_loader, train_loader, epochs=n_epoch, callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/20 Train steps: 48 Val steps: 48 900.71s loss: 6.548462 acc: 0.725354 val_loss: 5.358982 val_acc: 0.527530
Epoch 1: val_acc improved from -inf to 0.52753, saving file to freeze_layer1_norm_imagenet/best_epoch_1.ckpt
Epoch:  2/20 Train steps: 48 Val steps: 48 881.29s loss: 5.353803 acc: 0.527530 val_loss: 5.862512 val_acc: 0.923178
Epoch 2: val_acc improved from 0.52753 to 0.92318, saving file to freeze_layer1_norm_imagenet/best_epoch_2.ckpt
Epoch:  3/20 Train steps: 48 Val steps: 48 877.12s loss: 5.259434 acc: 0.857237 val_loss: 5.384838 val_acc: 0.989120
Epoch 3: val_acc improved from 0.92318 to 0.98912, saving file to freeze_layer1_norm_imagenet/best_epoch_3.ckpt
Epoch:  4/20 Train steps: 48 Val steps: 48 879.72s loss: 5.234078 acc: 0.989120 val_loss: 5.208830 val_acc: 1.417738
Epoch 4: val_acc improved from 0.98912 to 1.41774, saving file to freeze_layer1_norm_imagenet/best_epoch_4.ckpt
Epoch:  5/20 Train steps: 48 Val steps: 48 877.15s loss: 5.180451 acc: 0.923178 val_los

## modèle pré-entraîné, mais en laissant tous les paramètres se faire ajuster par backprop

In [70]:
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

In [71]:
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
callbacks = get_callbacks('no_freeze_norm_imagenet')

model = Model(resnet18, optimizer, loss_function, batch_metrics=['accuracy'],  device=device)

model.fit_generator(train_loader, train_loader, epochs=n_epoch, callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(test_loader)

Epoch:  1/20 Train steps: 48 Val steps: 48 953.42s loss: 6.475927 acc: 0.725354 val_loss: 5.300955 val_acc: 0.692384
Epoch 1: val_acc improved from -inf to 0.69238, saving file to no_freeze_norm_imagenet/best_epoch_1.ckpt
Epoch:  2/20 Train steps: 48 Val steps: 48 924.65s loss: 5.315629 acc: 0.692384 val_loss: 5.566242 val_acc: 1.121002
Epoch 2: val_acc improved from 0.69238 to 1.12100, saving file to no_freeze_norm_imagenet/best_epoch_2.ckpt
Epoch:  3/20 Train steps: 48 Val steps: 48 924.88s loss: 5.239985 acc: 1.121002 val_loss: 5.247614 val_acc: 2.011210
Epoch 3: val_acc improved from 1.12100 to 2.01121, saving file to no_freeze_norm_imagenet/best_epoch_3.ckpt
Epoch:  4/20 Train steps: 48 Val steps: 48 923.80s loss: 5.133133 acc: 1.516650 val_loss: 5.007825 val_acc: 2.406858
Epoch 4: val_acc improved from 2.01121 to 2.40686, saving file to no_freeze_norm_imagenet/best_epoch_4.ckpt
Epoch:  5/20 Train steps: 48 Val steps: 48 950.07s loss: 4.995890 acc: 2.176063 val_loss: 4.947532 val_